# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings 
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,orsa,54.5153,30.4053,281.41,60,100,4.67,BY,1712875413
1,1,tiksi,71.6872,128.8694,247.47,94,23,5.43,RU,1712875414
2,2,port-aux-francais,-49.3500,70.2167,279.15,85,100,2.46,TF,1712875414
3,3,ciudad lazaro cardenas,17.9583,-102.2000,300.45,63,0,3.69,MX,1712875416
4,4,yellowknife,62.4560,-114.3525,276.44,60,40,4.63,CA,1712875416


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(x="Lng", y="Lat", size="Humidity", alpha=0.5, hover_cols=["City", "Country"], geo=True, tiles="OSM", color="City")

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp'] > 21) & 
                                   (city_data_df['Max Temp'] < 27) &
                                   (city_data_df['Wind Speed'] < 4.5) &
                                   (city_data_df['Cloudiness'] == 0)].dropna()


# # Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,orsa,BY,54.5153,30.4053,60,
1,tiksi,RU,71.6872,128.8694,94,
2,port-aux-francais,TF,-49.3500,70.2167,85,
3,ciudad lazaro cardenas,MX,17.9583,-102.2000,63,
4,yellowknife,CA,62.4560,-114.3525,60,
...,...,...,...,...,...,...
546,zanjan,IR,36.6736,48.4787,81,
547,duong dong,VN,10.2167,103.9667,78,
548,rio san juan,DO,19.6439,-70.0763,83,
549,pitkyaranta,RU,61.5734,31.4792,99,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "category":"hotels",
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
     
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
orsa - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
bentiu - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
port douglas - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
broken hill - nearest hotel: No hotel found
jayapura - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
yaren - nearest hotel: No hotel found
shubarshi - nearest hotel: No hotel found
mogadishu - nearest hotel: No hotel found
iqaluit - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
lensk - nearest h

utrik - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
djenne - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
pontes e lacerda - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
gondar - nearest hotel: No hotel found
namibe - nearest hotel: No hotel found
thunder bay - nearest hotel: No hotel found
shar - nearest hotel: No hotel found
cesme - nearest hotel: No hotel found
lucheng - nearest hotel: No hotel found
dhuusamarreeb - nearest hotel: No hotel found
souillac - nearest hotel: No hotel found
labrador city - nearest hotel: No hotel found
chipinge - nearest hotel: No hotel found
holualoa - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
vega de alatorre - nearest hotel: No hotel found
lagunas - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
tasnad - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
ancud - nearest hotel: No hotel fou

lufkin - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
puerto armuelles - nearest hotel: No hotel found
corner brook - nearest hotel: No hotel found
hotan - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: No hotel found
bandar-e lengeh - nearest hotel: No hotel found
poronaysk - nearest hotel: No hotel found
bambanglipuro - nearest hotel: No hotel found
amahai - nearest hotel: No hotel found
lander - nearest hotel: No hotel found
shumikha - nearest hotel: No hotel found
sona - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
guarapari - nearest hotel: No hotel found
tsaratanana - nearest hotel: No hotel found
carbonear - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
kroya - nearest hotel: No hotel found
phnom penh - nearest hotel: No hotel found
acarau - nearest hotel: No hotel found
katima mulilo - nearest hotel: No hotel found
coyhaique - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
0,orsa,BY,54.5153,30.4053,60,No hotel found
1,tiksi,RU,71.6872,128.8694,94,No hotel found
2,port-aux-francais,TF,-49.3500,70.2167,85,No hotel found
3,ciudad lazaro cardenas,MX,17.9583,-102.2000,63,No hotel found
4,yellowknife,CA,62.4560,-114.3525,60,No hotel found
...,...,...,...,...,...,...
546,zanjan,IR,36.6736,48.4787,81,No hotel found
547,duong dong,VN,10.2167,103.9667,78,No hotel found
548,rio san juan,DO,19.6439,-70.0763,83,No hotel found
549,pitkyaranta,RU,61.5734,31.4792,99,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(x="Lng", y="Lat", size="Humidity",
alpha=0.5, hover_cols=["City","Hotel Name", "Country"],
geo=True, tiles="OSM", color="City")

# Display the map
plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)